# Capstone Project - TechMart

# Author: Yosef Emshani - November 22nd 2025

## Aim of this project
As explained in the README file of the /yosefemshani/SQL-Capstone-Project/ GitHub, a real-world data analysis challenge is simulated using a dataset from TechMart, a growing retail chain. All used datasets and tables can be found in this repository. Credits to Coursera for the simulated datasets.

## 1. Data Cleaning & Preparation

### 1.1 Connect to database and load tables

In [74]:
import sqlite3
import pandas as pd

# Setting up the database. DO NOT edit the code given below
from techsmart_db_setup import setup_database
setup_database() 
conn = sqlite3.connect('techsmart.db')

# Load and display tables
tables = ['Employee_Records', 'Product_Details', 'Customer_Demographics', 'Sales_Transactions']
for table in tables:
    query = f"SELECT * FROM {table} LIMIT 5"
    df = pd.read_sql_query(query, conn)
    print(f"\n{table}:")
    display(df)

✅ Database setup complete: Tables created and populated with data!

Employee_Records:


,employee_id,role,store_location,sales_performance
0,1,Cashier,New York,None
1,2,Cashier,Los Angeles,3000
2,3,Supervisor,Phoenix,None
3,4,Manager,Chicago,3000
4,5,Manager,Phoenix,3000



Product_Details:


,product_id,product_name,category,price,stock
0,106,Tablet,Electronics,one hundred,200.0
1,101,Keyboard,Accessories,fifty,100.0
2,108,Keyboard,Accessories,fifty,500.0
3,104,Speaker,Electronics,500,150.0
4,107,Speaker,Electronics,fifty,300.0



Customer_Demographics:


,customer_id,age,gender,location,loyalty_program
0,1,35,F,San Antonio,None
1,2,50,M,London,No
2,3,37,M,Austin,No
3,4,None,M,San Antonio,None
4,5,28,M,San Antonio,None



Sales_Transactions:


,transaction_id,customer_id,product_id,employee_id,quantity,total_amount,sale_date
0,1,13.0,105,2.0,4,90.0,2025-03-03 00:00:00
1,2,37.0,105,51.0,2,90.0,2025-03-08 00:00:00
2,3,5.0,106a,NaN,three,NaN,2025-03-01 00:00:00
3,4,35.0,109b,22.0,three,40.0,2025-03-04 00:00:00
4,5,15.0,101,45.0,2,NaN,2025-03-06 00:00:00


### 1.2 Choose table: 

In this case, we will focus on two out of these four tables. *'Employee_Records'* as well as *'Product_Details'* will be analyzed.

### 1.3 Lookout for errors in table in order to work on data integrity

#### Employee_Records

In [75]:
# Example for Employee_Records
query = """
SELECT
    employee_id, role, store_location, sales_performance
FROM
    Employee_Records
"""
df = pd.read_sql_query(query, conn)
display(df)

,employee_id,role,store_location,sales_performance
0,1,Cashier,New York,None
1,2,Cashier,Los Angeles,3000
2,3,Supervisor,Phoenix,None
3,4,Manager,Chicago,3000
4,5,Manager,Phoenix,3000
...,...,...,...,...
95,96,Manager,Phoenix,3000
96,97,Manager,London,1000
97,98,Manager,New York,2000
98,99,Sales Associate,Paris,3000


This table shows a total of 100 rows with 4 different variables / rows. Noticable are the *'None'* entries for *sales_performance*. The total of those are now investigated.

In [76]:
# Investigating missing sales for Employee_Records
query = """
SELECT COUNT(*) as total_rows,
       SUM(CASE WHEN sales_performance IS NULL OR sales_performance = 'nan' THEN 1 ELSE 0 END) as missing_sales
FROM Employee_Records
"""
df = pd.read_sql_query(query, conn)
display(df)

,total_rows,missing_sales
0,100,11


We now understand that out of 100 different employees, 11 employees seem to have a missing sales performance entry. These 11 are now focussed on. It is important to mention, that there still might be **other corrupt data** inside of this table, which has yet not been observed!

### 1.4 Data Cleaning

In [77]:
# Standardize sales_performance in Employee_Records
query = """
UPDATE
    Employee_Records
SET
    sales_performance = NULL
WHERE
    sales_performance = 'None';
"""
cursor = conn.cursor()
cursor.execute(query)
conn.commit()

In [78]:
# Print out Employee_Records to visualize result of data cleaning
query = """
SELECT
    sales_performance
FROM
    Employee_Records
"""
df = pd.read_sql_query(query, conn)
display(df)

,sales_performance
0,None
1,3000
2,None
3,3000
4,3000
...,...
95,3000
96,1000
97,2000
98,3000


At first sight, no changes have been made. However, this procedure made sure that no employees with the entry *'None' are NULL-ed. Now, let us investigate if these 11 were the only issue, or if there is any other corrupt data.

In [79]:
# Print out Employee_Records with text instead of a value, e.g. "fifty" instead of "50"
query = """
SELECT
    sales_performance
FROM
    Employee_Records
WHERE
    typeof(sales_performance) = 'text'
    AND length(
        replace(
        replace(
        replace(
        replace(
        replace(
        replace(
        replace(
        replace(
        replace(
        replace(sales_performance, '0', ''), '1', ''), '2', ''), '3', ''),
        '4', ''), '5', ''), '6', ''), '7', ''), '8', ''), '9', '')
    ) = length(sales_performance);

"""
df = pd.read_sql_query(query, conn)
display(df)

,sales_performance
0,five thousand
1,five thousand
2,five thousand
3,five thousand
4,five thousand
5,five thousand
6,five thousand
7,five thousand
8,five thousand
9,five thousand


In this case, there are 25 corrupt entries with text values, instead of integar values. This is now normalized in the following.

In [80]:
# Normalize values for sales_performance
query = """
UPDATE
    Employee_Records
SET
    sales_performance = CASE
    WHEN
        sales_performance = 'five thousand' THEN '5000'
    ELSE
        sales_performance
END
"""
cursor = conn.cursor()
cursor.execute(query)
conn.commit()

In [81]:
# Print out Employee_Records with text instead of a value, e.g. "fifty" instead of "50"
query = """
SELECT
    sales_performance
FROM
    Employee_Records
WHERE
    typeof(sales_performance) = 'text'
    AND length(
        replace(
        replace(
        replace(
        replace(
        replace(
        replace(
        replace(
        replace(
        replace(
        replace(sales_performance, '0', ''), '1', ''), '2', ''), '3', ''),
        '4', ''), '5', ''), '6', ''), '7', ''), '8', ''), '9', '')
    ) = length(sales_performance);

"""
df = pd.read_sql_query(query, conn)
display(df)

,sales_performance


As we can, two measurements have now been made to clean our dataset for further analysis. Now, the second table is investigated. Same steps are applied.

#### Product_Details

In [82]:
# Price for Product_Details
query = """
SELECT
    price
FROM
    Product_Details
"""
df = pd.read_sql_query(query, conn)
display(df)

,price
0,one hundred
1,fifty
2,fifty
3,500
4,fifty
...,...
95,300
96,25
97,500
98,fifty


At first glance, the same issue arises. Thus, now *'one hundred'* will be converted to *'100'* and *'fifty'* to *'50'*.

In [83]:
# Normalize price values
query = """
UPDATE
    Product_Details
SET
    price = CASE
    WHEN
        price = 'one hundred' THEN '100'
    WHEN
        price = 'fifty' THEN '50'
    ELSE
        price
END
"""
cursor = conn.cursor()
cursor.execute(query)
conn.commit()

In [84]:
# Print out prince for Product_Details with text instead of a value
query = """
SELECT
    price
FROM
    Product_Details
WHERE
    typeof(price) = 'text'
    AND length(
        replace(
        replace(
        replace(
        replace(
        replace(
        replace(
        replace(
        replace(
        replace(
        replace(price, '0', ''), '1', ''), '2', ''), '3', ''),
        '4', ''), '5', ''), '6', ''), '7', ''), '8', ''), '9', '')
    ) = length(price);

"""
df = pd.read_sql_query(query, conn)
display(df)

,price


As can be seen, our data is now ready for analysis.

## 2. Data Analysis

### 2.1 Employee Analysis

First of all, employees are ranked by their average sale performance. This is being taken account for the store location and employee count of this location, respectively.

In [85]:
query = """
WITH emp_sales AS (
    SELECT store_location, 
           AVG(CAST(sales_performance AS FLOAT)) as avg_sales,
           COUNT(*) as employee_count
    FROM Employee_Records
    WHERE sales_performance IS NOT NULL
    GROUP BY store_location
)
SELECT store_location, avg_sales, employee_count,
       RANK() OVER (ORDER BY avg_sales DESC) as location_rank
FROM emp_sales
ORDER BY avg_sales DESC
"""
df = pd.read_sql_query(query, conn)
display(df)

,store_location,avg_sales,employee_count,location_rank
0,Chicago,3875.000000,8,1
1,Phoenix,3312.500000,16,2
2,London,3214.285714,14,3
3,New York,2823.529412,17,4
4,Los Angeles,2692.307692,13,5
5,Paris,2600.000000,10,6
6,Houston,2363.636364,11,7


Noticable is that Chicago has the least amount of employees, yet showing the most amount of average sales.

### 2.2 Product Analysis

In the following, most saled product categories are demonstrated.

In [86]:
query = """
SELECT *
FROM (
    SELECT p.category,
           p.product_name,
           SUM(s.quantity) AS total_sold,
           RANK() OVER (
               PARTITION BY p.category
               ORDER BY SUM(s.quantity) DESC
           ) AS rank_in_category
    FROM Sales_Transactions s
    JOIN Product_Details p ON s.product_id = p.product_id
    GROUP BY p.category, p.product_name
)
WHERE rank_in_category <= 3
ORDER BY category, total_sold DESC
"""
df = pd.read_sql_query(query, conn)
display(df)

,category,product_name,total_sold,rank_in_category
0,Accessories,Keyboard,156.0,1
1,Accessories,Charger,123.0,2
2,Accessories,Speaker,85.0,3
3,Electronics,Tablet,134.0,1
4,Electronics,Mouse,129.0,2
5,Electronics,Headphones,110.0,3


It seems for accessories, that keyboards are the most sold, while for electronics, tablets are the most sold items.

These two examples of data analysis show how different aspects could be analyzed within a dataset and meaningful information can be extracted for companies to make **data-driven decisions**.

However, code can also be **optimized for run-time and general efficiency**. The goal is to get the same output of these two codes for analysis, while having a shorter run-time. AI can be a tool to support optimizing already working code. It is important to question the AI output however, as well as always test it before using it as a final code!

Generally, using more advanced SQL techniques, like **subqueries, CTEs, window functions or indexing** might be general ideas for optimization.

### 2.3 Optimized Employee Analysis

In [87]:
# Creating index for performance increase
cursor = conn.cursor()
cursor.execute("""
CREATE INDEX IF NOT EXISTS idx_employee_records_store_sales
ON Employee_Records (store_location, sales_performance)
""")
conn.commit()

In [88]:
query = """
WITH emp_sales AS (
    SELECT
        store_location,
        AVG(CAST(sales_performance AS FLOAT)) AS avg_sales,
        COUNT(*) AS employee_count
    FROM Employee_Records
    WHERE sales_performance IS NOT NULL
    GROUP BY store_location
)
SELECT
    store_location,
    avg_sales,
    employee_count,
    RANK() OVER (ORDER BY avg_sales DESC) AS location_rank
FROM emp_sales
ORDER BY avg_sales DESC
"""
df = pd.read_sql_query(query, conn)
display(df)

,store_location,avg_sales,employee_count,location_rank
0,Chicago,3875.000000,8,1
1,Phoenix,3312.500000,16,2
2,London,3214.285714,14,3
3,New York,2823.529412,17,4
4,Los Angeles,2692.307692,13,5
5,Paris,2600.000000,10,6
6,Houston,2363.636364,11,7


As we can see, the same output is delivered. Indexing leads to a more efficient way of this analysis, since the index needs to be run only once, while following subqueries can be adjusted as needed with already processed information using the index.

### 2.4 Optimized Product Analysis

Again, indexing would be a great idea to optimize this analysis as well.

In [89]:
# Creating first index for performance increase
cursor = conn.cursor()
cursor.execute("""
CREATE INDEX IF NOT EXISTS idx_sales_transactions_product
ON Sales_Transactions (product_id);
""")
conn.commit()

In [90]:
# Creating second index for performance increase
cursor = conn.cursor()
cursor.execute("""
CREATE INDEX IF NOT EXISTS idx_product_details_product_category
ON Product_Details (product_id, category);
""")
conn.commit()

In [91]:
query = """
WITH ranked_sales AS (
    SELECT
        p.category,
        p.product_name,
        SUM(s.quantity) AS total_sold,
        RANK() OVER (
            PARTITION BY p.category
            ORDER BY SUM(s.quantity) DESC
        ) AS rank_in_category
    FROM Sales_Transactions s
    JOIN Product_Details p
      ON s.product_id = p.product_id
    GROUP BY
        p.category,
        p.product_name
)
SELECT
    category,
    product_name,
    total_sold,
    rank_in_category
FROM ranked_sales
WHERE rank_in_category <= 3
ORDER BY
    category,
    total_sold DESC;
"""
df = pd.read_sql_query(query, conn)
display(df)

,category,product_name,total_sold,rank_in_category
0,Accessories,Keyboard,156.0,1
1,Accessories,Charger,123.0,2
2,Accessories,Speaker,85.0,3
3,Electronics,Tablet,134.0,1
4,Electronics,Mouse,129.0,2
5,Electronics,Headphones,110.0,3


The query is slightly improved while also using two indices for easier reproduction of changed variables within main calculating query, while delivering the same result.

## 3. Data Report

This section deals with possible ideas on how this project's data analysis can be summarized, as well as providing necessary information for the company to make data-driven decisions.

Generally, data needed to be cleaned due to values being not normalized. TechMart employees are advised to **focus on normalizing entry values in datasets**.

Furthermore, it has been shown that some employees seem to have **no sale performance listed**. A possible explanation for that would be, that these employees are **newly recruited** and have not made any sales yet. It is also possible, however, that entries have gone missing, which would need to be figured out with the staff concerning **data safety**.

Moreover, data analysis resulted in **Chicago having the highest amount of sales**, while **Houston having the least amount of sales**. This matter needs to be discussed with the corresponding staff in both cities to analyze where these differences might stem from and how sales generally can be **optimized** in Houston for example.

Additionally, **keyboards seem to be the most relevant item** in the category of accessories as well as **tablets being the most sold product** in electronics. This information could benefit the company to further emphasize on the current market in respect to these items. Perhaps **new merchandise** as well as **advertisments focussed on this fact** could be beneficial.

In a real-world setting, these kind of data reports need some sort of **visualization**, in order to better demonstrate differences between values, as well as help the team understand the analyzed points and results even more. Visualization techniques using **Python (Seaborn, Matplotlib + Pandas)**, **Tableau** or even **Power BI** will be analyzed as a following step of Data Analysis. Perhaps, the same dataset will be used for further analaysis, as seen in the public GitHub repository in the following weeks and months.